In [1]:
import os, sys
sys.path.append('/esat/opal/kkelchte/docker_home/tensorflow/q-learning/')

In [23]:
import os, sys
from pilot.model import Model
from pilot import data
import tensorflow as tf
import argparse
import skimage.io as sio
import skimage.transform as sm
import numpy as np


In [3]:
parser = argparse.ArgumentParser(description='Main pilot that can train or evaluate online or offline from a dataset.')
  
# ==========================
#   Training Parameters
# ==========================
parser.add_argument("--testing", action='store_true', help="In case we're only testing, the model is tested on the test.txt files and not trained.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Start learning rate.")
parser.add_argument("--batch_size",default=64,type=int,help="Define the size of minibatches.")

# ==========================
#   Offline Parameters
# ==========================
parser.add_argument("--offline", action='store_false', help="Training from an offline dataset.")
parser.add_argument("--max_episodes",default=1000,type=int,help="The maximum number of episodes (~runs through all the training data.)")

# ===========================
#   Utility Parameters
# ===========================
# Print output of ros verbose or not
parser.add_argument("--load_config", action='store_true',help="Load flags from the configuration file found in the checkpoint path.")
parser.add_argument("--verbose", action='store_false', help="Print output of ros verbose or not.")
parser.add_argument("--summary_dir", default='tensorflow/log/', type=str, help="Choose the directory to which tensorflow should save the summaries.")
parser.add_argument("--log_tag", default='testing', type=str, help="Add log_tag to overcome overwriting of other log files.")
parser.add_argument("--device", default='/gpu:0', type=str, help= "Choose to run on gpu or cpu: /cpu:0 or /gpu:0")
parser.add_argument("--random_seed", default=123, type=int, help="Set the random seed to get similar examples.")
parser.add_argument("--owr", action='store_true', help="Overwrite existing logfolder when it is not testing.")
parser.add_argument("--action_bound", default=1.0, type=float, help= "Define between what bounds the actions can go. Default: [-1:1].")
parser.add_argument("--action_dim", default=1.0, type=float, help= "Define the dimension of the actions: 1dimensional as it only turns in yaw.")
parser.add_argument("--real", action='store_true', help="Define settings in case of interacting with the real (bebop) drone.")
parser.add_argument("--evaluate", action='store_true', help="Just evaluate the network without training.")
parser.add_argument("--random_learning_rate", action='store_true', help="Use sampled learning rate from UL(10**-2, 1)")
parser.add_argument("--plot_depth", action='store_true', help="Specify whether the depth predictions is saved as images.")

# ===========================
#   Data Parameters
# ===========================
parser.add_argument("--normalize_data", action='store_true', help="Define wether the collision tags 0 or 1 are normalized in a batch. Only relevant for coll q net.")
parser.add_argument("--dataset", default="canyon_rl_turtle", type=str, help="pick the dataset in data_root from which your movies can be found.")
parser.add_argument("--data_root", default="~/pilot_data",type=str, help="Define the root folder of the different datasets.")
parser.add_argument("--num_threads", default=4, type=int, help="The number of threads for loading one minibatch.")
parser.add_argument("--collision_file", default='collision_info.txt', type=str, help="Define the name of the file with the collision labels.")
parser.add_argument("--control_file", default='control_info.txt', type=str, help="Define the name of the file with the action labels.")

# ===========================
#   Model Parameters
# ===========================
parser.add_argument("--depth_multiplier",default=0.25,type=float, help= "Define the depth of the network in case of mobilenet.")
parser.add_argument("--network",default='coll_q_net',type=str, help="Define the type of network: depth_q_net, coll_q_net.")
# parser.add_argument("--n_fc", action='store_true',help="In case of True, prelogit features are concatenated before feeding to the fully connected layers.")
# parser.add_argument("--n_frames",default=3,type=int,help="Specify the amount of frames concatenated in case of n_fc.")

# INITIALIZATION
parser.add_argument("--checkpoint_path",default='mobilenet_025', type=str, help="Specify the directory of the checkpoint of the earlier trained model.")
parser.add_argument("--continue_training",action='store_true', help="Continue training of the prediction layers. If false, initialize the prediction layers randomly.")
parser.add_argument("--scratch", action='store_true', help="Initialize full network randomly.")

# TRAINING
parser.add_argument("--weight_decay",default=0.00004,type=float, help= "Weight decay of inception network")
parser.add_argument("--init_scale", default=0.0005, type=float, help= "Std of uniform initialization")
parser.add_argument("--grad_mul_weight", default=0, type=float, help="Specify the amount the gradients of prediction layers.")
parser.add_argument("--dropout_keep_prob", default=0.5, type=float, help="Specify the probability of dropout to keep the activation.")
parser.add_argument("--clip_grad", default=0, type=int, help="Specify the max gradient norm: default 0 is no clipping, recommended 4.")
parser.add_argument("--min_depth", default=0.001, type=float, help="clip depth loss with weigths to focus on correct depth range.")
parser.add_argument("--max_depth", default=2.0, type=float, help="clip depth loss with weigths to focus on correct depth range.")
parser.add_argument("--optimizer", default='adadelta', type=str, help="Specify optimizer, options: adam, adadelta, gradientdescent, rmsprop")
# parser.add_argument("--no_batchnorm_learning",action='store_false', help="In case of no batchnorm learning, are the batch normalization params (alphas and betas) not further adjusted.")
parser.add_argument("--initializer",default='xavier',type=str, help="Define the initializer: xavier or uniform [-init_scale, init_scale]")

parser.add_argument("--loss",default='mse',type=str, help="Define the loss: mse, huber or absolute")

# ===========================
#   Replay Parameters
# ===========================

parser.add_argument("--normalized_replay", action='store_true', help="Make labels / actions equally likely for the coll / depth q net.")

# ===========================
#   Rosinterface Parameters
# ===========================
parser.add_argument("--buffer_size", default=1000, type=int, help="Define the number of experiences saved in the buffer.")
parser.add_argument("--ou_theta", default=0.05, type=float, help= "Theta is the pull back force of the OU Noise.")
parser.add_argument("--noise", default='ou', type=str, help="Define whether the noise is temporally correlated (ou) or uniformly distributed (uni).")
parser.add_argument("--sigma_z", default=0.0, type=float, help= "sigma_z is the amount of noise in the z direction.")
parser.add_argument("--sigma_x", default=0.0, type=float, help= "sigma_x is the amount of noise in the forward speed.")
parser.add_argument("--sigma_y", default=0.0, type=float, help= "sigma_y is the amount of noise in the y direction.")
parser.add_argument("--sigma_yaw", default=0.0, type=float, help= "sigma_yaw is the amount of noise added to the steering angle.")
parser.add_argument("--speed", default=0.5, type=float, help= "Define the forward speed of the quadrotor.")
parser.add_argument("--epsilon",default=0, type=float, help="Apply epsilon-greedy policy for exploration.")
parser.add_argument("--epsilon_decay", default=0.0, type=float, help="Decay the epsilon exploration over time with a slow decay rate of 1/10.")
parser.add_argument("--prefill", action='store_true', help="Fill the replay buffer first with random (epsilon 1) flying behavior before training.")

parser.add_argument("--action_amplitude", default=1, type=int, help="Define the action that is used as input to estimate Q value.")
parser.add_argument("--action_quantity",default=3, type=int, help="Define the number of actions used at the forward pass to evaluate a state.")
parser.add_argument("--action_smoothing",action='store_true', help="Define whether the actions should be sampled uniformly within the bin to represent better the continuous actions space.")
parser.add_argument("--action_normalization",action='store_true', help="Normalize action space from -1:1 to 0:1 avoiding gradients to cancel out.")

parser.add_argument("--off_policy",action='store_true', help="In case the network is off_policy, the control is published on supervised_vel instead of cmd_vel.")
parser.add_argument("--show_depth",action='store_false', help="Publish the predicted horizontal depth array to topic ./depth_prection so show_depth can visualize this in another node.")

parser.add_argument("--grad_steps", default=10, type=int, help="Define the number of batches or gradient steps are taken between 2 runs.")
parser.add_argument(
        '-f',
        '--file',
        help='Path for input file. First line should contain number of lines to search in'
    )
FLAGS=parser.parse_args()

In [7]:
# Set correct parameters for model you want to test
FLAGS.network='depth_q_net'
FLAGS.continue_training=True
FLAGS.checkpoint_path='/esat/opal/kkelchte/docker_home/tensorflow/log/off_depth_turtle/turtle_depth_abs_clip0001-2_001'

FLAGS.data_root="/esat/opal/kkelchte/docker_home/pilot_data"
FLAGS.dataset="canyon_rl_turtle_collision_free_epsilon03"
data.prepare_data(FLAGS,(model.input_size[1], model.input_size[2], model.input_size[3]))

/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00010_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00011_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00012_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00013_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00014_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00015_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00016_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00017_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00018_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00019_canyon
/esat/opal/kkelchte/

/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00096_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00097_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00098_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00099_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00100_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00101_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00102_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00103_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00104_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00105_canyon
/esat/opal/kkelchte/

In [5]:
config=tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = Model(FLAGS, sess)


checkpoint: /esat/opal/kkelchte/docker_home/tensorflow/log/off_depth_turtle/turtle_depth_abs_clip0001-2_001/2018-03-01_1818
Successfully loaded model from:/esat/opal/kkelchte/docker_home/tensorflow/log/off_depth_turtle/turtle_depth_abs_clip0001-2_001/2018-03-01_1818


In [32]:
for r in data.full_set['train']:
    num_random=0
    for i in range(len(r['num_imgs'])):
        img_file = os.path.join(r['name'],'RGB', '{0:010d}.jpg'.format(r['num_imgs'][i]))
        img = sio.imread(img_file)
        img = img[::2,::5,:]
        img = sm.resize(img,data.im_size,mode='constant').astype(float) #.astype(np.float32)
        inputs=np.asarray([img]*3)
        actions=np.asarray([-1,0,1]).reshape((-1,1))
        output, _=model.forward([img]*3, actions=actions)
        action = float(actions[np.argmax([np.amin(o[o!=0]) for o in output])])
        diff=np.abs(action-r['controls'][i])
        if diff > 0.7: num_random+=1
#         print("applied: {0}, predicted: {1}, diff: {2}".format(r['controls'][i],action,diff))
    print("num: {0} tot: {1} ~ {2}".format(num_random, len(r['num_imgs']), num_random/(0.+len(r['num_imgs']))))
    import pdb; pdb.set_trace()

num: 770 tot: 1902 ~ 0.40483701367
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 809 tot: 1885 ~ 0.429177718833
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 774 tot: 1900 ~ 0.407368421053
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 76 tot: 167 ~ 0.455089820359
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 427 tot: 1100 ~ 0.388181818182
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 189 tot: 451 ~ 0.419068736142
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 785 tot: 1901 ~ 0.412940557601
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train']:
(Pdb) c
num: 749 tot: 1897 ~ 0.394833948339
> <ipython-input-32-1f0622dc0c41>(1)<module>()
-> for r in data.full_set['train

BdbQuit: 

In [35]:
for i in range(10):
    print data.full_set['train'][i]['name']

/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00010_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00011_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00012_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00013_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00014_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00015_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00016_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00017_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00018_canyon
/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl_turtle_collision_free_epsilon03/00019_canyon
